<a href="https://colab.research.google.com/github/hackerinheels/dresser/blob/main/dresser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a notebook that builds a model to identify a traditional dresser against a contemporary or industrial style dresser.

In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
print(tf.__version__)

2.6.0


In [19]:
import pathlib
dataset_url = "https://storage.googleapis.com/furndata/data.zip"

data_dir = tf.keras.utils.get_file(origin=dataset_url, 
                                   fname='data', 
                                   untar=True)
data_dir = pathlib.Path(data_dir)

87498752/87488946 [==============================] - 1s 0us/step


In [ ]:
import glob
for name in data_dir.glob("*"):
    print(name)

In [20]:
!pwd
!ls -ltr /root/.keras/datasets/

/content
total 85440
-rw-r--r-- 1 root root 87488946 Sep 27 16:21 data.tar.gz


In [ ]:
# First run unzip data.tar.gz
!cd /root/.keras/datasets/
!unzip /root/.keras/datasets/data.tar.gz -d ./
print(len(list(data_dir.glob('*/*'))))


In [ ]:
traditional = list(data_dir.glob('*/*'))

In [25]:
!ls -tlr /root/.keras/datasets/
!cp /root/.keras/datasets/data.tar.gz /content/
!unzip /content/data.tar.gz

total 85440
-rw-r--r-- 1 root root 87488946 Sep 27 16:21 data.tar.gz
Archive:  /content/data.tar.gz
replace data/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [35]:
!ls -ltr /content/data/data

total 20
drwxr-xr-x 2 root root  4096 Sep 24 18:08 contemporary
drwxr-xr-x 2 root root 12288 Sep 24 18:09 traditional
drwxr-xr-x 2 root root  4096 Sep 24 18:10 industrial


In [36]:
data_dir = "/content/data/data"
data_dir = pathlib.Path(data_dir)

print(len(list(data_dir.glob('*/*'))))


686


In [37]:
batch_size = 32
img_height = 180
img_width = 180

In [115]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 683 files belonging to 3 classes.
Using 547 files for training.


In [86]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 683 files belonging to 3 classes.
Using 136 files for validation.


In [87]:
class_names = train_ds.class_names
print(class_names)

['contemporary', 'industrial', 'traditional']


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [44]:
IMG_SIZE = 180

resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
])
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [122]:
num_classes = 3
IMG_SIZE = 180
model = tf.keras.Sequential([
   
  tf.keras.layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, 3, activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(64, 3, activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(128, 3, activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [123]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [124]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

Epoch 1/10
18/18 [==============================] - 116s 6s/step - loss: 9.8388 - accuracy: 0.5192 - val_loss: 5.7834 - val_accuracy: 0.2941
Epoch 2/10
18/18 [==============================] - 114s 6s/step - loss: 4.9340 - accuracy: 0.5320 - val_loss: 2.0239 - val_accuracy: 0.5368
Epoch 3/10
18/18 [==============================] - 114s 6s/step - loss: 4.2861 - accuracy: 0.6143 - val_loss: 2.8597 - val_accuracy: 0.4926
Epoch 4/10
18/18 [==============================] - 115s 6s/step - loss: 1.5772 - accuracy: 0.6399 - val_loss: 1.9453 - val_accuracy: 0.4412
Epoch 5/10
18/18 [==============================] - 114s 6s/step - loss: 1.8290 - accuracy: 0.6874 - val_loss: 2.2698 - val_accuracy: 0.2426


Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7f33496a4710>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 546, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


Epoch 6/10
 3/18 [====>.........................] - ETA: 1:35 - loss: 0.8785 - accuracy: 0.7500

KeyboardInterrupt: ignored

In [ ]:
checkpoint_path = "cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=1,
  callbacks=[cp_callback]
)
# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

In [49]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
#loss, acc = model.evaluate(test_images, test_labels, verbose=2)


NOW Trying hyperparameterization automation using Keras Tuner with a diff model

In [125]:

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import BatchNormalization, experimental
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [162]:
# define cnn model
def build_model_1(hp):
	model = Sequential()
	#model.add(experimental.preprocessing.Rescaling(1./255))
	#model.add(experimental.preprocessing.Resizing(180, 180))
	hp_filters = hp.Int('filters', min_value = 32, max_value = 64, step = 32)
	model.add(Conv2D(filters=hp_filters, kernel_size=(3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(180,180, 3)))
	model.add(BatchNormalization())
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(BatchNormalization())
	model.add(Dropout(0.5))
	model.add(Dense(3, activation='softmax'))
	# compile model
	#opt = SGD(lr=0.001, momentum=0.9)
	hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3])
	opt = Adam(learning_rate=hp_learning_rate)#, momentum=0.9)
	model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
	model.summary()
	return model

In [163]:

tuner = RandomSearch(
    build_model_1,
    objective='val_accuracy',
    max_trials=4,
    executions_per_trial=1)
#,
#    directory='my_dir1') #change the directory name here  when rerunning the cell else it gives "Oracle exit error" 


INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 180, 180, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 180, 180, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 180, 180, 32)      9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 180, 180, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 90, 90, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 259200)            0         
____________________________________

In [147]:
!\rm -rf ./untitled_project/oracle.json

In [100]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [164]:
tuner.search(train_ds, validation_data=val_ds,
  epochs=3)

Trial 3 Complete [00h 07m 07s]
val_accuracy: 0.5220588445663452

Best val_accuracy So Far: 0.5220588445663452
Total elapsed time: 00h 17m 20s
INFO:tensorflow:Oracle triggered exit


In [165]:
tuner.results_summary()


Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
filters: 32
learning_rate: 0.01
Score: 0.5220588445663452
Trial summary
Hyperparameters:
filters: 64
learning_rate: 0.001
Score: 0.5220588445663452
Trial summary
Hyperparameters:
filters: 64
learning_rate: 0.01
Score: 0.41911765933036804


In [62]:
train_labels = np.concatenate([y for x, y in train_ds], axis=0)
val_labels = np.concatenate([y for x, y in val_ds], axis=0)


In [52]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 97 kB 2.9 MB/s 
